In [1]:
import torch
from torch import optim
from torchvision import datasets, transforms
import dnnlib
import torch_utils
import pickle

from copy import deepcopy

import stylegan2.training.dorm_networks as networks

import matplotlib.pyplot as plt
import pandas as pd
import json
import os
import numpy as np

from tqdm import tqdm
import tqdm as T
import random
tqdm.pandas()

import shutil

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
stylegan = pickle.load(open("./pretrained_stylegans/ffhq.pkl","rb"))

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
generator = stylegan['G']
G = networks.DoRMGenerator(generator.z_dim,generator.c_dim,generator.w_dim,generator.img_resolution,generator.img_channels).to(device) 
_ = G.load_state_dict(generator.state_dict(),strict = False)
D = stylegan['D'].to(device)

del generator

In [5]:
G.requires_grad_(False)
G.target_mapping.requires_grad_(True)
for x, i in G.named_modules():
    if 'target_affine' in x:
        i.requires_grad_(True)

D.requires_grad_(False)
for i in range(2,6):
    getattr(D,f'b{2**i}').requires_grad_(False)

In [6]:
LR = 1e-3
opt_g = optim.Adam([{'params': [param for name, param in G.named_parameters() if 'target_affine' in name]},
                     {'params': G.target_mapping.parameters(), 'lr': 1e-5}], lr=LR, betas =(0.0, 0.99))
opt_d = optim.Adam(
    D.parameters(), lr= LR, betas =(0.0, 0.99)
)

In [7]:
target_folder = "./DoRM/anime_faces/"
im_names = None
for x in os.walk(target_folder):
    im_names = x[2]

In [8]:
trainsform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.Resize((1024, 1024)),
     transforms.ToTensor(),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.Normalize(
        [0.5 for _ in range(3)],
        [0.5 for _ in range(3)],
     )
    ]
)

data = torch.Tensor(np.array([trainsform(torch.from_numpy(plt.imread(os.path.join(target_folder,x))).permute(2,0,1)).numpy() for x in im_names])).to(device)

/tmp/ipykernel_3685403/2328931497.py:13: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:178.)
  data = torch.Tensor(np.array([trainsform(torch.from_numpy(plt.imread(os.path.join(target_folder,x))).permute(2,0,1)).numpy() for x in im_names])).to(device)


In [ ]:
counter = 0
total_samples = 100000
with tqdm(total=total_samples) as pbar:
    while counter < total_samples:
        target_batch = data[[random.randint(0,9) for _ in range(4)]]

        zs = torch.randn(4, G.z_dim).to(device)
        zt = torch.randn(4, G.z_dim).to(device)
        c = torch.zeros(4,G.c_dim).to(device)
        
        # generate images from both source & target domains
        gen_target = G(zs, zt, c, 1.0)
        # gen_source = G(zs, zt, c, 0.0)
        
        disc_target = D(target_batch,c)
        disc_gen = D(gen_target,c)
        # disc_source = D(gen_source.detach(),c)
        
        # Critic loss is loss for 
        loss_disc = -torch.mean(1 - disc_gen) - torch.mean(disc_target)
        # loss_disc = -torch.mean(1 - disc_gen) - torch.mean(disc_target) + torch.mean(disc_source)
        D.zero_grad()
        loss_disc.backward()
        opt_d.step()
        
        gen_target = G(zs, zt, c, 1.0)
        disc_gen = D(gen_target,c)
        
        loss_gen = -torch.mean(disc_gen)
        loss_gen.backward()
        opt_g.step()

        counter+=4
        pbar.update(4)

  1%|▉                                                                                                                                                                      | 528/100000 [01:46<5:36:23,  4.93it/s]